In [7]:
import os
import zipfile
import numpy as np
import librosa
import tensorflow as tf
from google.colab import files, drive
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# ==== STEP 1: UPLOAD DATASET ====

# Choose dataset upload method
UPLOAD_METHOD = "drive"  # Options: "drive" (Google Drive) or "manual" (local upload)

if UPLOAD_METHOD == "drive":
    drive.mount('/content/drive')
    DATASET_PATH = "/content/drive/MyDrive/recordingss"  # Update with actual path
    EXTRACT_PATH = "/content/dataset"

    # Unzip Dataset
    if not os.path.exists(EXTRACT_PATH):
        os.makedirs(EXTRACT_PATH)
    !unzip -q "$DATASET_PATH" -d "$EXTRACT_PATH"
    print("✅ Dataset extracted to", EXTRACT_PATH)

elif UPLOAD_METHOD == "manual":
    print("📂 Upload your dataset ZIP file")
    uploaded = files.upload()
    for filename in uploaded.keys():
        if filename.endswith(".zip"):
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall("/content/dataset")
    EXTRACT_PATH = "/content/dataset"
    print("✅ Dataset extracted!")

# Check files
print("📁 Files in dataset:", os.listdir(EXTRACT_PATH))


Mounted at /content/drive
unzip:  cannot find or open /content/drive/MyDrive/recordingss, /content/drive/MyDrive/recordingss.zip or /content/drive/MyDrive/recordingss.ZIP.
✅ Dataset extracted to /content/dataset
📁 Files in dataset: []


In [8]:
RECORDINGS_FOLDER ="/content/drive/MyDrive/recordingss"  # Use extracted dataset folder
N_MFCC = 20
MAX_TIME_STEPS = 50

# Function to extract MFCC features
def extract_mfcc(audio_file, max_time_steps=MAX_TIME_STEPS, n_mfcc=N_MFCC):
    y, sr = librosa.load(audio_file, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
  # Ensure consistent shape (20, 50)
    if mfcc.shape[1] < max_time_steps:
        pad_width = max_time_steps - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0, 0), (0, pad_width)), mode='constant')
    elif mfcc.shape[1] > max_time_steps:
        mfcc = mfcc[:, :max_time_steps]

    return mfcc.T  # Shape (max_time_steps, n_mfcc)


In [9]:
# Extract features & labels
X, y = [], []
for audio_file in os.listdir(RECORDINGS_FOLDER):
    if audio_file.endswith('.m4a'):
        label = 1 if "aut_" in audio_file else 0  # Assuming naming convention
        mfcc = extract_mfcc(os.path.join(RECORDINGS_FOLDER, audio_file))
        X.append(mfcc)
        y.append(label)

X = np.array(X)  # Shape: (samples, max_time_steps, n_mfcc)
y = to_categorical(y, num_classes=2)  # Convert labels to categorical


<ipython-input-8-bc003070b4e7>:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file, sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-8-bc003070b4e7>:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file, sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-8-bc003070b4e7>:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file, sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/cor

In [10]:
# ==== STEP 2: PREPROCESS AUDIO DATA ====
RECORDINGS_FOLDER = EXTRACT_PATH  # Use extracted dataset folder
N_MFCC = 20
MAX_TIME_STEPS = 50


In [11]:
# Function to extract MFCC features
def extract_mfcc(audio_file, max_time_steps=MAX_TIME_STEPS, n_mfcc=N_MFCC):
    y, sr = librosa.load(audio_file, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)

    # Ensure consistent shape (20, 50)
    if mfcc.shape[1] < max_time_steps:
        pad_width = max_time_steps - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0, 0), (0, pad_width)), mode='constant')
    elif mfcc.shape[1] > max_time_steps:
        mfcc = mfcc[:, :max_time_steps]

    return mfcc.T  # Shape (max_time_steps, n_mfcc)


In [12]:
from sklearn.model_selection import train_test_split

# Check if X and y exist
print(f"✅ Total Dataset Shape: {X.shape}, Labels Shape: {y.shape}")

# Ensure X and y are properly defined before splitting
if 'X' in locals() and 'y' in locals():
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    print(f"✅ Training Data Shape: {X_train.shape}")
    print(f"✅ Testing Data Shape: {X_test.shape}")
else:
    print("❌ Error: X or y is not defined. Check dataset loading.")


✅ Total Dataset Shape: (43, 50, 20), Labels Shape: (43, 2)
✅ Training Data Shape: (34, 50, 20)
✅ Testing Data Shape: (9, 50, 20)


In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# ✅ Define LSTM Model
model = Sequential([
    LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3, input_shape=(50, 20)),  # First LSTM layer
    LSTM(64, return_sequences=True, dropout=0.3, recurrent_dropout=0.3),  # Second LSTM layer
    LSTM(32, dropout=0.3, recurrent_dropout=0.3),  # Third LSTM layer
    Dense(64, activation='relu'),  # Fully connected layer
    Dropout(0.3),  # Dropout to prevent overfitting
    Dense(2, activation='softmax')  # Output layer (2 classes)
])

# ✅ Compile Model (You need this before training)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0003)  # Adjust learning rate here
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])



/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
# Train Model
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# Save Model
model.save(MODEL_FILE)
print(f"✅ Model saved as {MODEL_FILE}")


Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.4844 - loss: 0.6965 - val_accuracy: 0.5556 - val_loss: 0.6774
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 417ms/step - accuracy: 0.6976 - loss: 0.6443 - val_accuracy: 0.5556 - val_loss: 0.6734
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 283ms/step - accuracy: 0.5823 - loss: 0.6644 - val_accuracy: 0.6667 - val_loss: 0.6710
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 236ms/step - accuracy: 0.6507 - loss: 0.6686 - val_accuracy: 0.7778 - val_loss: 0.6696
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 238ms/step - accuracy: 0.6204 - loss: 0.6621 - val_accuracy: 0.7778 - val_loss: 0.6677
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 235ms/step - accuracy: 0.7123 - loss: 0.6682 - val_accuracy: 0.7778 - val_loss: 0.6665
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 223ms/step - accuracy: 0.5607 - loss: 0.6794 - val_accuracy: 0.6667 - val_loss: 0.6665
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 230ms/step - accuracy: 0.5087 - loss: 0.6851 - val_accuracy: 0.6667 - val_loss: 0

✅ Model saved as /content/lstm_model.h5


In [15]:
MODEL_FILE = "/content/lstm_model.h5"
model.save(MODEL_FILE)  # Save model
print(f"✅ Model saved successfully at: {MODEL_FILE}")


✅ Model saved successfully at: /content/lstm_model.h5


In [16]:
# ==== STEP 4: TEST MODEL WITH NEW AUDIO ====
print("📂 Upload an .m4a file for testing")
uploaded = files.upload()
test_file = list(uploaded.keys())[0]

# Load trained model
model = tf.keras.models.load_model(MODEL_FILE)

# Extract MFCC features from test file
mfcc_features = extract_mfcc(test_file)

# Reshape for LSTM input
mfcc_features = np.expand_dims(mfcc_features, axis=0)  # Shape (1, 50, 20)

# Predict Class
prediction = model.predict(mfcc_features)
predicted_label = "Autistic" if np.argmax(prediction) == 1 else "Non-Autistic"

print(f"🧠 Prediction: {predicted_label}")


📂 Upload an .m4a file for testing


Saving Recording (7).m4a to Recording (7).m4a


<ipython-input-11-704faf7ea42c>:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file, sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
🧠 Prediction: Non-Autistic


NameError: name 'model' is not defined

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model saved in Google Drive: /content/drive/MyDrive/lstm_model.h5
